In [1]:
import sys 
sys.path.append('../')

import torch
import matplotlib.pyplot as plt 
from einops import repeat

from src.green_net import GreenNet2D
from src.model import MLP
from src.utils import rl2_error

In [2]:
def DiskPoisson(pts):
    x1 = pts[...,0]
    y1 = pts[...,1]
    x2 = pts[...,2]
    y2 = pts[...,3]

    mask = ((x1**2+y1**2) < 1) & ((x2**2+y2**2) < 1)

    k = 1/(4*torch.pi) * torch.log(
        ((x1 - x2)**2 + (y1-y2)**2) / \
        ((x1*y2-x2*y1)**2 + (x1*x2+y1*y2-1)**2))
    k = torch.nan_to_num(k, neginf=-1) * mask

    return k

In [3]:
def DiskInvDisk(pts):
    x1 = pts[...,0]
    y1 = pts[...,1]
    x2 = pts[...,2]
    y2 = pts[...,3]

    mask = ((x1**2+y1**2) < 1) & ((x2**2+y2**2) < 1)

    k = ((x1 - x2)**2 + (y1-y2)**2) ** (-0.5)
    k = torch.nan_to_num(k, posinf=50) * mask

    return k

In [4]:
def ffunc_2D(pts):
    x = pts[:,0]
    y = pts[:,1]
    u = 1 - (x**2+y**2)
    u = torch.nan_to_num(u, posinf=0)
    return u

In [5]:
device = torch.device('cpu')
# kernel = MLP([2, 64, 1], nonlinearity='relu').to(device)
# model = GreenNet2D(n=6, kernel=DiskPoisson, device=device, sub_num=5)
model = GreenNet2D(n=6, kernel=DiskInvDisk, device=device, sub_num=5)

In [6]:
F = ffunc_2D(model.grid.x_h)
F = repeat(F, 'n -> b n', b=64)

Ground Truth Kernel Function

In [7]:
model.eval_K()
nh = model.grid.nh
K_full = model.K_hh.reshape(nh,nh,nh,nh)

In [8]:
xy = model.grid.x_h.reshape(nh,nh,2)
mask = (xy[...,0]**2 + xy[...,1]**2) > 1
G_xy00 = K_full[:,:,32,32]
# G_xy00[mask] = torch.nan
G_x0y0 = K_full[:,32,:,32]
plt.subplot(121)
plt.pcolor(xy[...,0], xy[...,1], G_xy00, cmap='jet')
plt.title('$G(x,y,0,0)$')
plt.gca().set_aspect('equal')
plt.colorbar()
plt.subplot(122)
plt.pcolor(xy[...,0], xy[...,1], G_x0y0, cmap='jet')
plt.title('$G(x,0,x\',0)$')
plt.gca().set_aspect('equal')
plt.colorbar()
plt.tight_layout()

Full Kernel Integral

In [9]:
F = ffunc_2D(model.grid.x_h)
F = repeat(F, 'n -> b n', b=64)
U_full = model.full_kint(F)
f = F[0].reshape(nh,nh)
u = U_full[0].reshape(nh,nh)
f[mask] = torch.nan 
u[mask] = torch.nan

/tmp/ipykernel_4103190/4254651839.py:5: MatplotlibDeprecationWarning: Getting the array from a PolyQuadMesh will return the full array in the future (uncompressed). To get this behavior now set the PolyQuadMesh with a 2D array .set_array(data2d).
  plt.colorbar()
/tmp/ipykernel_4103190/4254651839.py:10: MatplotlibDeprecationWarning: Getting the array from a PolyQuadMesh will return the full array in the future (uncompressed). To get this behavior now set the PolyQuadMesh with a 2D array .set_array(data2d).
  plt.colorbar()


In [10]:
plt.subplot(121)
plt.pcolor(xy[...,0], xy[...,1], f)
plt.title('input function F')
plt.gca().set_aspect('equal')
plt.colorbar()
plt.subplot(122)
plt.pcolor(xy[...,0], xy[...,1], u)
plt.title('output function U')
plt.gca().set_aspect('equal')
plt.colorbar()
plt.tight_layout()

Test Eval Batch

In [11]:
F = ffunc_2D(model.grid.x_h)
F = repeat(F, 'n -> b n', b=64)
model.rand_sub()
print('random sub : ', model.sub)
model.eval_K_sub()
U_sub = model.sub_kint(F)

random sub :  tensor([3562, 1372, 2404, 3489, 1169])


In [12]:
U_full[:,model.sub][0]

tensor([1.9851, 2.2363,    nan, 2.5741,    nan])

In [13]:
U_sub[0]

tensor([1.9851, 2.2363, 0.0000, 2.5741, 0.0000])

Test Batch Kint

In [14]:
F = ffunc_2D(model.grid.x_h)
F = repeat(F, 'n -> b n', b=64)
model.eval_K()
U_full = model.full_kint(F)
U_batch = model.evalint_batch(F)

In [17]:
F.shape

torch.Size([64, 4225])

In [16]:
U_batch.shape

torch.Size([64, 4225])

In [15]:
print('U rl2 : ' , rl2_error(U_batch, U_full))

U rl2 :  tensor(1.4409e-07)
